In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
import time
import os
import numpy as np
from pathlib import Path
from PIL import Image
from skimage.transform import resize
import helper
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.image import imread
from torchvision import datasets
from torchvision import datasets, transforms, models
from torch import nn, optim, Tensor
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from torch.utils.data import DataLoader, Dataset, TensorDataset
import random
seed=42

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

In [ ]:
base_path = Path('../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud')
red_dir   = base_path/'train_red_additional_to38cloud'
blue_dir  = base_path/'train_blue_additional_to38cloud'
green_dir = base_path/'train_green_additional_to38cloud'
nir_dir   = base_path/'train_nir_additional_to38cloud'
gt_dir    = base_path/'train_gt_additional_to38cloud'

In [ ]:
class RGB_CloudDataset (Dataset):
    def __init__(self, red_dir, blue_dir, green_dir, gt_dir, transform= None):
        
        
        self.transform   = transform
    
        
        # Listing subdirectories
        # Loop through the files in red folder  
        # and combine, into a dictionary, the other bands
        
        self.files = [self.combine_files(f, green_dir, blue_dir, gt_dir) 
                      for f in red_dir.iterdir() if not f.is_dir()]
        
        random.seed (seed)
        self.files = random.sample (self.files, k= 8000)
        
        
    def combine_files(self, red_file: Path, green_dir, blue_dir, gt_dir):
        
        files = {'red': red_file, 
                 'green':green_dir/red_file.name.replace('red', 'green'),
                 'blue': blue_dir/red_file.name.replace('red', 'blue'), 
                 'gt': gt_dir/red_file.name.replace('red', 'gt')}

        return files
    

    
    
    def OpenAsArray(self, idx):
        
        TrueColor = np.stack([np.array(Image.open(self.files[idx]['red'])),
                              np.array(Image.open(self.files[idx]['green'])),
                              np.array(Image.open(self.files[idx]['blue']))], axis = 2)
     
    
    
        TrueColor = TrueColor.transpose((2, 0, 1))
    
        return TrueColor 
    
    
    
    def OpenMask(self, idx, add_dims=False):
        raw_mask=np.array(Image.open(self.files[idx]['gt']))
        raw_mask = np.where(raw_mask==255, 1, 0)
        
        return np.expand_dims(raw_mask, 0) if add_dims else raw_mask


        
    def __len__(self):
        return len(self.files)
    
    
    def __getitem__(self, idx):
        x = self.OpenAsArray(idx)
        y = self.OpenMask(idx, add_dims=False)
        
        
        if self.transform is not None:
            x, y = self.transform((x, y))
        
        
        return torch.from_numpy(x), torch.from_numpy(y)
    
    
    def open_as_pil(self, idx):
        arr = self.OpenAsArray(idx)
        
        return Image.fromarray(arr.astype(np.uint8), 'RGB')  
    
    
    
    def __repr__(self):
        s = 'Dataset class with {} files'.format(self.__len__())

        return s

In [ ]:
class NirGB_CloudDataset (Dataset):
    def __init__(self, red_dir, blue_dir, green_dir, nir_dir, gt_dir, transform = None):
        
        
        self.transform = transform
        
        # Listing subdirectories
        # Loop through the files in red folder  
        # and combine, into a dictionary, the other bands
        
        self.files = [self.combine_files(f, green_dir, blue_dir, nir_dir, gt_dir) 
                      for f in red_dir.iterdir() if not f.is_dir()]
        
        
        
    def combine_files(self, red_file: Path, green_dir, blue_dir, nir_dir, gt_dir):
        
        files = {'red': red_file, 
                 'green':green_dir/red_file.name.replace('red', 'green'),
                 'blue': blue_dir/red_file.name.replace('red', 'blue'), 
                 'nir': nir_dir/red_file.name.replace('red', 'nir'),
                 'gt': gt_dir/red_file.name.replace('red', 'gt')}

        return files
    
    
    
    def OpenAsArray(self, idx):
        
        FalseColor = np.stack([np.array(Image.open(self.files[idx]['nir'])),
                               np.array(Image.open(self.files[idx]['green'])),
                               np.array(Image.open(self.files[idx]['blue']))], axis = 2)
     
                    
        FalseColor      = FalseColor.transpose((2, 0, 1))
    
        return (FalseColor / np.iinfo(FalseColor.dtype).max)
    
    
    
    def OpenMask(self, idx, add_dims=False):
        raw_mask=np.array(Image.open(self.files[idx]['gt']))
        raw_mask = np.where(raw_mask==255, 1, 0)
        
        return np.expand_dims(raw_mask, 0) if add_dims else raw_mask


        
    def __len__(self):
        return len(self.files)
    
    
    def __getitem__(self, idx):
        x = self.OpenAsArray(idx)
        y = self.OpenMask(idx, add_dims=False)
        return torch.from_numpy(x), torch.from_numpy(y)
    
    
    def open_as_pil(self, idx):
        arr = 256 * self.OpenAsArray(idx)
        return Image.fromarray(arr.astype(np.uint8), 'NirGB')  
    
    
    
    def __repr__(self):
        s = 'Dataset class with {} files'.format(self.__len__())
        return s

In [ ]:
class Resize(object):
    def __init__(self, size = 256):
        self.size = size
    def __call__(self, sample):
        x, y = sample
        return (resize(x, (x.shape[0], self.size, self.size), mode = "constant", 
                      preserve_range = True, anti_aliasing = False),
                resize(y, (self.size, self.size), mode = "constant", 
                      preserve_range = True, anti_aliasing = False))
    

class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std  = std
        
    def __call__(self, sample):
        x,y =sample
        x = x.transpose(1,2,0)
        x=(x-self.mean)/self.std
        return x.transpose(2,0,1), y
    
    
# TODO: Define transforms for the training data and testing data
train_transforms=transforms.Compose([Resize(256),
                                     Normalize([0.485, 0.456, 0.406], [0.229, 0.224,0.225])])

                                 
test_transforms=transforms.Compose(Normalize([0.485, 0.456, 0.406], [0.229, 0.224,0.225]))

In [ ]:
RGB_data   = RGB_CloudDataset(red_dir, blue_dir, green_dir, gt_dir, transform = train_transforms) 
NirGB_data = NirGB_CloudDataset(red_dir, blue_dir, green_dir, nir_dir, gt_dir, transform = None)
RGB_x, RGB_y = RGB_data[1000]
NirGB_x, NirGB_y = NirGB_data[1000]
RGB_x.shape, RGB_y.shape, NirGB_x.shape, NirGB_y.shape

In [ ]:
# splitting the data into train, validation, and test datasets

RGBtrain_size = int(0.75 * len(RGB_data))
RGBvalid_size = int(0.15 * len(RGB_data))
RGBtest_size  = len(RGB_data) - RGBtrain_size - RGBvalid_size
RGBremaining_size = len(RGB_data) - RGBtrain_size 

RGBtrain_dataset, RGBremaining_dataset = torch.utils.data.random_split(RGB_data, 
                                                                       [RGBtrain_size, RGBremaining_size])
RGBvalid_dataset, RGBtest_dataset      = torch.utils.data.random_split(RGBremaining_dataset, 
                                                                       [RGBvalid_size, RGBtest_size])


print('\t\t\tDataset')
print("Train data: \t\t{}".format(len(RGBtrain_dataset)),
      "\nValidation data: \t{}".format(len(RGBvalid_dataset)),
     "\nTest data: \t\t{}".format(len(RGBtest_dataset)))



RGBtrain_loader = DataLoader(RGBtrain_dataset, batch_size=12, shuffle=True, num_workers=2)
RGBvalid_loader = DataLoader(RGBvalid_dataset, batch_size=12, shuffle=True, num_workers=2)
RGBtest_loader  = DataLoader(RGBtest_dataset , batch_size=12, shuffle=True, num_workers=2)

RGBdata_iter = iter(RGBvalid_loader)
rgb_img, mask = next(RGBdata_iter)

print('\n')
print('Raw RGB image shape on batch size = {}'.format(rgb_img.size()))
print('Cloud Mask shape on batch size    = {}'.format(mask.size()))

## **Model: Pretrained Vgg16**

In [ ]:
class Vgg16(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=True)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(256,256), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[0].parameters():
            param.require_grad = False
        for param in self.modified_network[1:].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
model = Vgg16().to(device)
model.modified_network [0], model.modified_network [1:]

In [ ]:
#checking if we have the correct dimensions for our (3,384,384) images
#summary(Vgg16, (3, 384, 384))   

# checking if the network works, using one batch of the training set
rgb_img, mask = next(iter(RGBtrain_loader))
rgb_img, mask = rgb_img.to(device, dtype=torch.float), mask.to(device) 
output = model (rgb_img).to(device)
output.shape, mask.shape 

In [ ]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(model.parameters(), lr=0.0001)


def acc_fn(predb, yb):
    predb = torch.sigmoid(predb)
    return (predb.argmax(dim=1) == yb.to(device)).float().mean()

In [ ]:
def train(model, train_dl, valid_dl, loss_fn, optimizer, acc_fn, epochs, save_path):
    
    start = time.time()
    min_valid_epoch_loss = np.inf
    best_acc = 0.0
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    for e in range(epochs):
        print('Epoch {}'.format(e))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to train mode
                dataloader = train_dl
            else:
                model.eval()  # Set model to evaluate mode
                dataloader = valid_dl

            running_loss = 0.0
            running_acc  = 0.0


            # iterate over data
            for x, y in dataloader:
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.int64)

                # forward pass 
                if phase == 'train':
                    # zero the gradients
                    optimizer.zero_grad()
                    outputs = model(x)
                    loss    = loss_fn(outputs, y)
                    # backward + optimize only if in training phase
                    # the backward pass frees the graph memory, so there is no 
                    # need for torch.no_grad in this training pass
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()

                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss    = loss_fn(outputs, y.long())

                # stats - whatever is the phase
                acc = acc_fn(outputs, y)
                running_acc  += acc * dataloader.batch_size
                running_loss += loss.item() * dataloader.batch_size
                    
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc  = running_acc / len(dataloader.dataset)

            print('{} \tLoss: {:.4f} \tAcc: {}'.format( phase, epoch_loss, epoch_acc))
            print()

            train_losses.append(epoch_loss), train_accuracies.append(epoch_acc) if phase=='train' else valid_losses.append(epoch_loss), valid_accuracies.append(epoch_acc)

             # save model if validation loss has decreased
            if phase == 'valid':
                if epoch_loss <= min_valid_epoch_loss:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                        min_valid_epoch_loss, epoch_loss)) 
                    print('Best Validation Accuracy ({:.6f} --> {:.6f}).'.format(
                        best_acc, epoch_acc))
                    torch.save(model.state_dict(), save_path)
                    min_valid_epoch_loss = epoch_loss
                    best_acc = epoch_acc
            

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_losses, valid_losses, train_accuracies, valid_accuracies, best_acc

In [ ]:
def plot_accuracies(train_accuracies, valid_accuracies):
    plt.plot(train_accuracies, '-bx')
    plt.plot(valid_accuracies, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    
def plot_losses(train_losses, valid_losses):
    plt.plot(train_losses, '-bx')
    plt.plot(valid_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
model.freeze()
Pretrained_Vgg16 = './Pretrained_Vgg16_CloudSegModel.pt'
train_losses, valid_losses, train_accuracies, valid_accuracies, best_acc = train(model,RGBtrain_loader, RGBvalid_loader, loss_fn, optimizer, acc_fn, epochs=25, save_path = Pretrained_Vgg16)

In [ ]:
plot_accuracies(train_accuracies, valid_accuracies)
plt.savefig('./Accuracies_plot_Pretrained_Vgg16.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Pretrained_Vgg16.png')

## **Model: Untrained Vgg16**

In [ ]:
class Vgg16_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=False)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(256,256), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)

In [ ]:
model_np = Vgg16_np().to(device)
model_np.modified_network [0], model_np.modified_network [1:]

In [ ]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(model_np.parameters(), lr=0.0001)


def acc_fn(predb, yb):
    predb = torch.sigmoid(predb)
    return (predb.argmax(dim=1) == yb.to(device)).float().mean()

Untrained_Vgg16 = './Untrained_Vgg16_CloudSegModel.pt'
train_losses, valid_losses, train_accuracies, valid_accuracies, best_acc= train(model_np, RGBtrain_loader, RGBvalid_loader, 
                                             loss_fn, optimizer, acc_fn, epochs=25, save_path = Untrained_Vgg16)

In [ ]:
plot_accuracies(train_accuracies, valid_accuracies)
plt.savefig('./Accuracies_plot_Untrained_Vgg16.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Untrained_Vgg16.png')

## **Model: Pretrained Resnet34**

In [ ]:
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.resnet34(pretrained=True)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(256,256), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[:8].parameters():
            param.require_grad = False
        for param in self.modified_network[8].parameters():
            param.require_grad = True
        for param in self.modified_network[9].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
ResNet_model = ResNet34().to(device)
ResNet_model.modified_network[8], ResNet_model.modified_network[9]

In [ ]:
# checking if the network works, using one batch of the training set
rgb_img, mask = next(iter(RGBtrain_loader))
rgb_img, mask = rgb_img.to(device, dtype=torch.float), mask.to(device) 
output = ResNet_model (rgb_img).to(device)
output.shape, mask.shape 

In [ ]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(ResNet_model.parameters(), lr=0.0001)


def acc_fn(predb, yb):
    predb = torch.sigmoid(predb)
    return (predb.argmax(dim=1) == yb.to(device)).float().mean()

In [ ]:
ResNet_model.freeze()
Pretrained_ResNet34 = './Pretrained_ResNet34_CloudSegModel.pt'
train_losses, valid_losses, train_accuracies, valid_accuracies, best_acc = train(ResNet_model,RGBtrain_loader, RGBvalid_loader, loss_fn, optimizer, acc_fn, epochs=25, save_path = Pretrained_ResNet34)

In [ ]:
plot_accuracies(train_accuracies, valid_accuracies)
plt.savefig('./Accuracies_plot_Pretrained_ResNet34.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Pretrained_ResNet34.png')

## **Model: Untrained Resnet34**

In [ ]:
class ResNet34_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=False)
        # Replace the classifier
        self.network = nn.Sequential(*list((*list(self.network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(256,256), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
Resnet_np = ResNet34_np().to(device)
Resnet_np.network[8], Resnet_np.network[9]

In [ ]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(Resnet_np.parameters(), lr=0.0001)


def acc_fn(predb, yb):
    predb = torch.sigmoid(predb)
    return (predb.argmax(dim=1) == yb.to(device)).float().mean()

Untrained_ResNet34 = './Untrained_ResNet34_CloudSegModel.pt'
train_losses, valid_losses, train_accuracies, valid_accuracies, best_acc= train(Resnet_np, RGBtrain_loader, RGBvalid_loader, 
                                             loss_fn, optimizer, acc_fn, epochs=25, save_path = Untrained_ResNet34)

In [ ]:
plot_accuracies(train_accuracies, valid_accuracies)
plt.savefig('./Accuracies_plot_Untrained_ResNet34.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Untrained_ResNet34.png')